In [1]:
import http.client
import urllib.request
import urllib.parse
import urllib.error
import base64
import json
import os

from IPython.display import Image
import cv2  
import pandas as pd
import numpy as np

## Load tagged from custom vision

In [2]:
headers = {
 # Request headers
 'Training-Key': '1a4a4c41bd394819be757c43ac8f1931' #--------------------แก้ตาม Custom Project
}
project_id = "2bdfc083-fefa-40dc-a48e-1f86af540ecd" #--------------------แก้ตาม Custom Project

In [3]:
params = urllib.parse.urlencode({
 # Format - int32. Maximum number of images to return. Defaults to 50, limited to 256.
 'take': '50',
 # Format - int32. Number of images to skip before beginning the image batch. Defaults to 0.
 'skip': '0'
})

In [4]:
try:
     # base url
    conn = http.client.HTTPSConnection('southeastasia.api.cognitive.microsoft.com')
    conn.request("GET", f"/customvision/v3.0/training/projects/{project_id}/images/tagged?{params}", "{body}", headers)
    response = conn.getresponse() 
    # get response as a raw string
    data = response.read() 
    # convert the string to a json object
    data_json = json.loads(data)
except:
    print('error')

In [5]:
len(data_json)

10

## Dataframe for checking images

In [6]:
df = pd.read_csv('/media/tohn/SSD/mammo_custom/mammo.csv')
df.head(5)

,Unnamed: 0,Case,FileName,Path,Split,Category
0,0,127,mammo_127.jpg,/media/tohn/SSD/mammo_custom/test/mammo_127.jpg,test,test
1,1,123,mammo_123.jpg,/media/tohn/SSD/mammo_custom/test/mammo_123.jpg,test,test
2,2,121,mammo_121.jpg,/media/tohn/SSD/mammo_custom/test/mammo_121.jpg,test,test
3,3,128,mammo_128.jpg,/media/tohn/SSD/mammo_custom/test/mammo_128.jpg,test,test
4,4,124,mammo_124.jpg,/media/tohn/SSD/mammo_custom/test/mammo_124.jpg,test,test


In [7]:
len(df)

129

In [8]:
thisdf=df

### Checking images loop 

#### รอบที่ 1 : 50

In [9]:
def findmat (thisdf):
    for record in thisdf.iterrows():
        img_path = record[1]['Path']
        img_c = cv2.imread(img_path)
        try:
            if (img_c.shape[0] == img.shape[0]) and (img_c.shape[1] == img.shape[1]):
                img_dif = np.sum(img-img_c)
                path = 'None'
                if img_dif == 0:
                    print(img_path)
                    path = img_path
                return path
        except:
            path = 'None'
            print('this is none')

In [10]:
datadf = list()
for i in range(0,len(data_json)):
    
    #load image origin save to test.jpg
    urllib.request.urlretrieve(data_json[i]['originalImageUri'],'test.jpg')
    img = cv2.imread('test.jpg') 
    
    originalImage = data_json[i]['originalImageUri']
    for j in range(len(data_json[i]['regions'])):    
        tagName = data_json[i]['regions'][j]['tagName']
        left = int(data_json[i]['regions'][j]['left'] * data_json[i]['width'])
        top = int(data_json[i]['regions'][j]['top'] * data_json[i]['height'])
        width = int(data_json[i]['regions'][j]['width'] * data_json[i]['width'])
        height = int(data_json[i]['regions'][j]['height'] * data_json[i]['height'])
    #     img = cv2.rectangle(img, (left,top), (left+width,top+height), (255,0,0), 5)
        #Real point
        Rleft =data_json[i]['regions'][j]['left']
        Rtop = data_json[i]['regions'][j]['top']
        Rwidth = data_json[i]['regions'][j]['width']
        Rheight = data_json[i]['regions'][j]['height']
    
        #loop check img
        print(i)
        path = findmat(thisdf)
        print(path)

        datadf.append({'img_path':path,'tagName':tagName,'originalImage': originalImage, 'left':left, 'top':top,
                              'width':width,'height':height, 'Rleft':Rleft, 'Rtop':Rtop,
                              'Rwidth':Rwidth,'Rheight':Rheight})

0
None
0
None
1
None
1
None
1
None
2
None
2
None
3
None
3
None
3
None
4
/media/tohn/SSD/mammo_custom/test/mammo_120.jpg
/media/tohn/SSD/mammo_custom/test/mammo_120.jpg
4
/media/tohn/SSD/mammo_custom/test/mammo_120.jpg
/media/tohn/SSD/mammo_custom/test/mammo_120.jpg
4
/media/tohn/SSD/mammo_custom/test/mammo_120.jpg
/media/tohn/SSD/mammo_custom/test/mammo_120.jpg
4
/media/tohn/SSD/mammo_custom/test/mammo_120.jpg
/media/tohn/SSD/mammo_custom/test/mammo_120.jpg
4
/media/tohn/SSD/mammo_custom/test/mammo_120.jpg
/media/tohn/SSD/mammo_custom/test/mammo_120.jpg
5
/media/tohn/SSD/mammo_custom/test/mammo_125.jpg
/media/tohn/SSD/mammo_custom/test/mammo_125.jpg
5
/media/tohn/SSD/mammo_custom/test/mammo_125.jpg
/media/tohn/SSD/mammo_custom/test/mammo_125.jpg
5
/media/tohn/SSD/mammo_custom/test/mammo_125.jpg
/media/tohn/SSD/mammo_custom/test/mammo_125.jpg
5
/media/tohn/SSD/mammo_custom/test/mammo_125.jpg
/media/tohn/SSD/mammo_custom/test/mammo_125.jpg
5
/media/tohn/SSD/mammo_custom/test/mammo_125.jp

In [11]:
df = pd.DataFrame(datadf)
df.head()

,img_path,tagName,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight
0,None,mammo,https://irisprodseatraining.blob.core.windows....,599,282,50,40,0.841292,0.397099,0.070225,0.056180
1,None,mammo,https://irisprodseatraining.blob.core.windows....,599,518,37,25,0.841292,0.728560,0.053371,0.035112
2,None,mammo,https://irisprodseatraining.blob.core.windows....,70,380,50,47,0.099719,0.534739,0.070225,0.067416
3,None,mammo,https://irisprodseatraining.blob.core.windows....,102,333,42,34,0.144663,0.468728,0.058989,0.047753
4,None,mammo,https://irisprodseatraining.blob.core.windows....,39,334,52,44,0.054775,0.470133,0.074438,0.061798


In [12]:
print(len(df))

28


In [13]:
os.chdir('/home/yupaporn/USAI_labeling/')
df.to_csv('mammo_Test.csv')

### check null 

In [13]:
len(set(df['img_path']))

4

In [14]:
df[df['img_path'].isnull()]

,img_path,tagName,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight


In [15]:
df.loc[df['img_path'].isnull(),'img_path'] = 'None'

In [16]:
df[df['img_path']=='None']

,img_path,tagName,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight
0,None,mammo,https://irisprodseatraining.blob.core.windows....,599,282,50,40,0.841292,0.397099,0.070225,0.056180
1,None,mammo,https://irisprodseatraining.blob.core.windows....,599,518,37,25,0.841292,0.728560,0.053371,0.035112
2,None,mammo,https://irisprodseatraining.blob.core.windows....,70,380,50,47,0.099719,0.534739,0.070225,0.067416
3,None,mammo,https://irisprodseatraining.blob.core.windows....,102,333,42,34,0.144663,0.468728,0.058989,0.047753
4,None,mammo,https://irisprodseatraining.blob.core.windows....,39,334,52,44,0.054775,0.470133,0.074438,0.061798
5,None,mammo,https://irisprodseatraining.blob.core.windows....,363,245,60,53,0.511236,0.345133,0.084270,0.075843
6,None,mammo,https://irisprodseatraining.blob.core.windows....,439,561,65,47,0.616573,0.788953,0.091292,0.067416
7,None,mammo,https://irisprodseatraining.blob.core.windows....,286,238,36,36,0.401685,0.335301,0.051966,0.051966
8,None,mammo,https://irisprodseatraining.blob.core.windows....,303,456,42,31,0.426966,0.641481,0.058989,0.044944
9,None,mammo,https://irisprodseatraining.blob.core.windows....,392,299,93,63,0.550562,0.420975,0.132022,0.088483


### Save .csv 

In [27]:
os.chdir('/home/yupaporn/USAI_labeling/')
df.to_csv('AB01.csv')

In [31]:
# df[:142]['img_path'][141]

'/media/tohn/HDD/VISION_dataset/USAI/ABnormal01/4 ABNORMAL/cropped/AB04 P3-1 C024.JPG'

In [32]:
# df[:142]['img_path'][140]

'/media/tohn/HDD/VISION_dataset/USAI/ABnormal01/4 ABNORMAL/cropped/AB04 P1 C029.JPG'

In [33]:
# df[:142]['img_path'][139]

'/media/tohn/HDD/VISION_dataset/USAI/ABnormal01/4 ABNORMAL/cropped/AB04 P1 C029.JPG'

In [ ]:
# urllib.request.urlretrieve(data_json[0]['originalImageUri'],'test.jpg')
#     #Image(filename='test.jpg') 
# img = cv2.imread('test.jpg') 
    
    
# i = 0
# originalImage = data_json[i]['originalImageUri']
# tagName = data_json[i]['regions'][0]['tagName']
# left = int(data_json[i]['regions'][0]['left'] * data_json[i]['width'])
# top = int(data_json[i]['regions'][0]['top'] * data_json[i]['height'])
# width = int(data_json[i]['regions'][0]['width'] * data_json[i]['width'])
# height = int(data_json[i]['regions'][0]['height'] * data_json[i]['height'])
    
# img = cv2.rectangle(img, (left,top), (left+width,top+height), (255,0,0), 5)
# img_path = findmat (thisdf)
# Rleft =data_json[i]['regions'][0]['left']
# Rtop = data_json[i]['regions'][0]['top']
# Rwidth = data_json[i]['regions'][0]['width']
# Rheight = data_json[i]['regions'][0]['height']